# Função - Classificador

Está função será chamada pelo botão "Processar" da inteface.

Ela irá treinar um modelo com nossa base já classificada. Posteriormente irá pegar o excel selecioando e irá classificá-lo.

In [1]:
import pandas as pd
import re
from textblob import TextBlob
from tqdm.auto import tqdm

In [2]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import f1_score, recall_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score

### Função de preparo as manhcetes

In [3]:
# Pegar só as manchetes e limpar 
# igual está no DATASET_PREPARATION!!!
dataset = pd.read_excel("base_pronta.xlsx")

Por enquanto irei fazer considerando apenas o modelo SVM

In [ ]:
def cleanup(text):
    import string
    punctuation = '[\/!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [ ]:
def prepara_novo_data_set(dataset):
    
    #------------------------------------------------------------------
    manchetes = {}
    contador = 0
    email=[]
    lista_manchetes=[]
    numero_atual = 0
    loop = tqdm(total = data.shape[0], position = 0, leave = False)
    for i in data.HTML.index:
        loop.set_description("Extraindo manchetes...".format(numero_atual))
        loop.update(1)
        numero_atual += 1
        texto = str(data.HTML[i])

        for indice in range(len(texto)):

            if texto[indice:indice+7] == "message":

                manchete_incial = texto[indice+11:indice+150]
                achou = False
                manchete = " "
                indice_letra = 0
                email.append(i+2)

                while indice_letra < len(manchete_incial):

                    if manchete_incial[indice_letra] == "}":

                        manchete = manchete_incial[0:indice_letra-6]
                        achou = True
                        manchetes[contador] = manchete

                        manchete_limpa = cleanup(manchete)
                        manchete_min = manchete_limpa.lower()

                        try:
                            hi_blob = TextBlob(manchete_min)
                            manchete_pronta = hi_blob.translate(to='en')
                        except:
                            manchete_pronta = manchete_min

                        lista_manchetes.append(str(manchete_pronta))
                        contador += 1
                        break

                    else:

                        indice_letra += 1
    loop.close()
    #------------------------------------------------------------------
    lista_links = []
    for linha in data.HTML.index:   #Percorre cada linha do dataset
        texto = data.HTML[linha]

        contador_de_titulos = 0
        for indice in range(len(texto)-7):
            if texto[indice:indice+7] == "message":
                contador_de_titulos += 1



        indice = 0
        texto_com_url = ""

        while indice < len(texto):  #Percorre o texto dentro da linha
            if texto[indice:indice + 7] == "widgets":
                while True:
                    if texto[indice+5:indice+8] == "} ]":
                        break
                    else:
                        texto_com_url += texto[indice+8]
                        indice += 1
                break 
            else:
                indice += 1


        indice = 0
        numero_atual_de_titulos = 0

        while indice < len(texto_com_url) and numero_atual_de_titulos < contador_de_titulos:
            if texto_com_url[indice : indice+3] == "url":
                link = ""
                while True:
                    if texto_com_url[indice+7] == '"':
                        numero_atual_de_titulos += 1
                        break
                    else:
                        link += texto_com_url[indice+7]
                        indice += 1
                lista_links.append(link)
            else:
                indice += 1
    #------------------------------------------------------------------
    indice_lista = 0
    while indice_lista < len(lista_links):
        contador_de_https = 0
        contador = 0
        link = lista_links[indice_lista]
        while contador < len(link):
            if link[contador:contador+5] == "https" or link[contador:contador+4] == "http":
                contador_de_https += 1
            if contador_de_https == 2:
                link_completo = link[contador:len(link)]
                link_final = ""
                indice_google = 0
                while indice_google < len(link_completo):

                    if link_completo[indice_google : indice_google+7] == "u0026ct":
                        break
                    else:
                        link_final += link_completo[indice_google]
                        indice_google += 1

                lista_links[indice_lista] = link_final
                break
            contador += 1

        indice_lista += 1
    #------------------------------------------------------------------
    lista_backup = []
    
    loop = tqdm(total = relevantes.shape[0], position = 0, leave = False)
    numero_atual = 0
    for i in relevantes.index:
        loop.set_description("Manchetes base OBC...".format(numero_atual))
        loop.update(1)
        numero_atual += 1

        manchete_relevante = str(relevantes["Manchetes da Base de Dados"][i])
        manchete_relevante_limpa = cleanup(manchete_relevante)
        manchete_relevante_min = manchete_relevante_limpa.lower()

        try:
            manchete_relevante_blob = TextBlob(manchete_relevante_min)
            manchete_relevante_traduzida = manchete_relevante_blob.translate(to='en')
        except:
            manchete_relevante_traduzida = manchete_relevante_min

        lista_manchetes.append(str(manchete_relevante_traduzida))
        lista_backup.append(str(manchete_relevante_traduzida))
        lista_links.append("Não se aplica")

    loop.close()
    
    dicionario = {}
    
    dicionario['Manchetes'] = lista_manchetes
    #dicionario['Indice'] = email
    dicionario['Link'] = lista_links
    
    data_manchetes = pd.DataFrame(data=dicionario)
    
    result = pd.concat([data, data_manchetes], ignore_index=True, axis=1)
    
    result.columns = ['Data','De','HTML','Resumo','Manchete','Link']
    
    result["Relevância"] = int(0)
    
    data_backup = pd.read_excel("data.xlsx", sheet_name="backup")
    
    #------------------------------------------------------------------
    loop = tqdm(total = data_backup.shape[0], position = 0, leave=False)
    numero_atual = 0 
    for i in data_backup.index:
        loop.set_description("Adicionando manchetes relevantes...".format(numero_atual))
        loop.update(1)

        numero_atual += 1 


        manchete_d = str(data_backup.Manchetes[i])
        manchete_d_limpa = cleanup(manchete_d)
        manchete_d_min = manchete_d_limpa.lower()

        try:
            manchete_d_min_blob = TextBlob(manchete_d_min)
            manchete_d_traduzida = manchete_d_min_blob.translate(to='en')
        except:
            manchete_d_traduzida = manchete_d_min

        lista_backup.append(str(manchete_d_traduzida))

    loop.close()
    
    rel = 0
    for i in result.index:
        if result["Manchete"][i] in lista_backup:
            result.Relevância[i] = int(1)
            rel += 1
            
    return result

In [9]:
# Pegando as manchestes do arquivo novo
def classifica_novo_data_set(dataset, modelo):
    dicionario = dict()
    
    for linha in dataset.index:
        manchete = dataset["Manchete"][linha]
        relevancia = int(modelo.predict(manchete))
        
        if relevancia == 1:
            manchetes_relevantes.append(manchete)
            links_relevantes.append(dataset["Link"][linha])
    
    dicionario['Manchetes'] = manchetes_relevantes
    #dicionario['Indice'] = email
    dicionario['Link'] = links_relevantes
    
    data_pandas = pd.DataFrame(data=dicionario)
    
    return data_pandas
        
    

In [ ]:
def classificar (treino, arquivo):
    
    # Pegando as manchetes e suas respctivas relevâncias 
    X_para_treinar = treino.loc[:, "Manchete"]
    Y_para_treinar = treino.loc[:, "Relevância"]
    ##y = arquivo.loc[:, "Relevância"]
    
    # Vetorizando
    count_vect = CountVectorizer()
    X_train_counts = count_vect.fit_transform(X_para_treinar)

    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    
    # Sepando em teste e treino
    # Porém vou colocar tudo para treino
    X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, Y_para_treinar, test_size=0, random_state=42)
    
    #Aplicando o modelo SVM (Support Vector Machine)

    kernels = ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']

    model = svm.SVC(kernel = 'linear') #Kernel

    #Dando fit no modelo SVM usando o dataset de treino
    model.fit(X_train, y_train)
 # -------------------------------------------------------------------------------------------------------------   
    #Tratando o dataset escolhido
    dataset_escolhido_preparado = prepara_novo_data_set(arquivo)
    
    # Pegando as manchestes do arquivo novo
    planilha_pandas = classifica_novo_data_set(dataset_escolhido_preparado, model)
    
    # Tranformando em Excel
    planilha = planilha_pandas.to_excel (r'base_classificada.xlsx ', index = False)
    
    
    
    

    
    